Домашнняя работа по теме

Задание 1

In [ ]:
import requests
from bs4 import BeautifulSoup
import re
from typing import List
import pandas as pd

__all_posts: List[dict] = []
KEYWORDS = ['python', 'парсинг']

def read_data_from_remote_resource(url):
    return BeautifulSoup(requests.get(url).text, 'html.parser')

def load_available_pages(habr_all_page):
    def find_pagination_elements(html_page):
        return list(map(lambda x: x.find_all('a'), html_page.find_all('ul', id='nav-pagess')))[0]
    def map_pagination_elements_to_pages(elements_list):
        return list(map(lambda element: element.attrs.get('href'), elements_list))
    return map_pagination_elements_to_pages(find_pagination_elements(habr_all_page))

def find_last_page_num(available_pages_list):
    return int(list(map(lambda x: re.findall('\d+', x), available_pages_list))[len(available_pages_list) - 1][0])

def extract_posts_from_page_content(html_page):
    def extract_posts_content_block(habr_all_posts_page):
        return habr_all_posts_page.find_all('div', class_='posts_list')
    def extract_only_posts(habr_posts_block):
        return list(map(lambda element: element.find_all('article', class_='post_preview'), habr_posts_block))[0]
    return extract_only_posts(extract_posts_content_block(html_page))

def get_post_content(post: BeautifulSoup):
    def get_title(post_data: BeautifulSoup):
        return post_data.find_all('a', class_='post__title_link')[0].text
    def get_content(post_data: BeautifulSoup):
        return post_data.find_all('div', class_='post__text')[0].text
    def get_link(post_data: BeautifulSoup):
        return post_data.find_all('a', class_='post__title_link')[0].attrs.get('href')
    return {
        'title': get_title(post),
        'content': get_content(post),
        'link': get_link(post)
    }

def _is_contains_keywods(post_dict: dict):
    for keyword in KEYWORDS:
        if keyword.lower() in str(post_dict.get('title')).lower() \
                or keyword.lower() in str(post_dict.get('content')).lower():
            return True
    return False

def add_posts_from_collection(posts_collection: list):
    for post in posts_collection:
        _add_post(post)

def _add_post(post: BeautifulSoup):
    post_dict = get_post_content(post)
    if _is_contains_keywods(post_dict):
        __all_posts.append(post_dict)

first_page = read_data_from_remote_resource('https://habr.com/ru/all/')
max_page_number = find_last_page_num(load_available_pages(first_page))

add_posts_from_collection(extract_posts_from_page_content(first_page))

for i in range(2, max_page_number + 1):
    page = read_data_from_remote_resource('https://habr.com/ru/all/page' + str(i) + '/')
    add_posts_from_collection(extract_posts_from_page_content(page))

pd.DataFrame(__all_posts)

Задание 2

In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from typing import List
import json

EMAIL = ['xxx@x.ru', 'yyy@y.com']
# EMAIL = ['nik.turchin@yandex.ru']

def load_breaches(email: List[str]):
    return requests.post(
        url='https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data',
        json={ 'emailAddresses': email },
        headers={
            'Vaar-Version': '0',
            'Vaar-Header-App-Product': 'hackcheck-web-avast'
        }
    )

def decode_response(content: bytes):
    resp = json.loads(content)
    return resp.get('breaches'), resp.get('data'), resp.get('summary')

def __get_breach(list: dict, id: int):
    return list.get(str(id))

def get_source(breaches_list: dict, breach_id: int):
    return __get_breach(breaches_list, breach_id).get('site')

def get_details(breaches_list: dict, breach_id: int):
    return __get_breach(breaches_list, breach_id).get('description')

def get_breach_date(breaches_list: dict, breach_id: int):
    return __get_breach(breaches_list, breach_id).get('publishDate')

response = load_breaches(EMAIL)
breaches, _, summary = decode_response(response.content)

__all_breaches = []

for email, details in summary.items():
    __all_breaches.extend(list(map(
        lambda breach_id: {
            'email': email,
            'source': get_source(breaches, breach_id),
            'published_at': get_breach_date(breaches, breach_id),
            'details': get_details(breaches, breach_id)
        },
        details.get('breaches')
    )))


pd.DataFrame(__all_breaches)